## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

data = pd.read_csv('data/Churn.csv')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
round(data.isna().mean() * 100) 

RowNumber          0.0
CustomerId         0.0
Surname            0.0
CreditScore        0.0
Geography          0.0
Gender             0.0
Age                0.0
Tenure             9.0
Balance            0.0
NumOfProducts      0.0
HasCrCard          0.0
IsActiveMember     0.0
EstimatedSalary    0.0
Exited             0.0
dtype: float64

Проанализировав пропущенные значения, было установлено, что только в столбце Tenure (Сколько лет клиент сотрудничает с банком) есть пропуски, процент которых составляет 9% от всего датасета. Таким образом, мы можем исключить данные строки из выборки.

In [6]:
data = data.dropna(subset = ['Tenure'])

In [7]:
# Убедимся что все пропуски удалены
round(data.isna().mean() * 100) 

RowNumber          0.0
CustomerId         0.0
Surname            0.0
CreditScore        0.0
Geography          0.0
Gender             0.0
Age                0.0
Tenure             0.0
Balance            0.0
NumOfProducts      0.0
HasCrCard          0.0
IsActiveMember     0.0
EstimatedSalary    0.0
Exited             0.0
dtype: float64

In [8]:
# Приводим Tenure к типу int
data['Tenure'] = data['Tenure'].astype(int)

In [9]:
# Необходимо удалить столбцы, которые не нужны для создания модели
data = data.drop(columns = ['RowNumber', 'CustomerId', 'Surname'])

Выше указанные столбцы необходимо было удалить, так как модель на основе этих данных не могла бы сделать прогноз, они не несут в себе важной информации.

In [10]:
# Необходимо привести столбцы к общему стилю
data = data.rename(
    columns={'CreditScore' : 'Credit_score',
             'NumOfProducts' : 'Num_of_products',
             'HasCrCard' : 'Has_сr_сard',
             'IsActiveMember' : 'Is_active_member',
             'EstimatedSalary' : 'Estimated_salary'}
)

In [11]:
data.duplicated().sum()

0

In [12]:
data_ohe = pd.get_dummies(data, drop_first = True)
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis = 1)

# Необходимо разбить данные на обучающую, валидационную и тестовую выборку (3:1:1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size = 0.4, random_state = 12345, stratify = target) 
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size = 0.5, random_state = 12345, stratify = target_valid)

In [13]:
# Размер выборки
display(features_valid.shape)
display(target_valid.shape)

(1818, 11)

(1818,)

In [14]:
# Размер выборки
display(features_test.shape)
display(target_test.shape)

(1819, 11)

(1819,)

На этапе предобработке данных были выявлены и устранены ошибки в общем стиле данных, неверный формат / тип данных, обнаружены пропущенные значения. Краткий анализ показал, что явные дубликаты в датасете отсутствуют, в целом данные чистые. Также данные были разбиты на обучающую, валидационную и  тестовую выборку в соотношении 3:1:1. 

## Исследование задачи

In [15]:
data['Exited'].value_counts()

0    7237
1    1854
Name: Exited, dtype: int64

Был произведён анализ целевого признака на предмет сбалансированности. Было установлено, что соотношение классов несбалансированно и составляет примерно 80 / 20.

In [16]:
# Построим модель случайного леса
best_model = None
best_f1 = 0
best_depth = 0
best_est = 0

for est in range(10, 100, 10):
    for depth in range(1,30):
        model = RandomForestClassifier(n_estimators = est, random_state = 12345, max_depth = depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_model = model
            best_f1 = f1
            best_depth = depth
            best_est = est


In [17]:
print('best n_estimators =', best_est, '; ', 
      'best max_depth =', best_depth, '; ', 
      'best f1 = ', round(best_f1, 4)
     ) 

best n_estimators = 50 ;  best max_depth = 20 ;  best f1 =  0.6234


Проведя исследование была выявлена лучшая модель случайного леса по метрики f1, у которой данный показатель составил 0,6234. Оптимальные гиперпараметры глубина\количество деревьев составили 20 и 50 соотвественно.

In [18]:
probabilities_valid = best_model.predict_proba(features_valid) # Заменила на best_model, auc_roc немного уменьшился
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(round(auc_roc,4))

0.8654


In [19]:
# Построим модель дерево решений
best_model = None
best_f1 = 0
best_depth = 0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
#    print("Глубина дерева", depth, ":", f1)
    if f1 > best_f1:
        best_model = model
        best_f1 = f1
        best_depth = depth

In [20]:
print('best max_depth =', best_depth, '; ', 
      'best f1 = ', round(best_f1, 4)
     ) 

best max_depth = 7 ;  best f1 =  0.5815


Построив модель дерева решений было установлено, что метрика f1 принимает лучшее значение (0,5815) на глубине - 7. 
f1 = 0.5815

In [21]:
probabilities_valid = best_model.predict_proba(features_valid) # Заменила на best_model, auc_roc значительно вырос
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(round(auc_roc,4))

0.8245


Таким образом, на данном этапе для несбалансированной выборки модель случайного леса продемонстрировала лучшее показатели метрики f1 и auc_roc - 0.6234 и 0.8654 соотвественно. 

## Борьба с дисбалансом

**Взвешивание классов**

In [22]:
balanced_model = DecisionTreeClassifier(
    random_state = 12345, max_depth = 6, class_weight = 'balanced') # Был подобран новый оптимальный параметр depth = 6
balanced_model.fit(features_train, target_train)
predicted_valid = balanced_model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print('f1:', round(f1,4))

f1: 0.5831


In [23]:
probabilities_valid = balanced_model.predict_proba(features_valid) 
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(round(auc_roc,4))

0.8354


In [24]:
balanced_model = RandomForestClassifier(n_estimators = 90, random_state = 12345, max_depth = 9, class_weight = 'balanced')
balanced_model.fit(features_train, target_train)
predicted_valid = balanced_model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print('f1:', round(f1,4))

f1: 0.6386


In [25]:
probabilities_valid = balanced_model.predict_proba(features_valid) 
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(round(auc_roc,4))

0.8756


После метода взвешивания классов модели со старыми гиперпараметрами демонстрировали примерно схожий результат. f1 = 0.56. Данные модели не удовлетворяли поставленным изначальным требованиям (минимальный порог f1 = 0.59). Именно поэтому для уже сбалансированной выборки были найдены новые оптимальные параметры: для дерева решений max_depth составила 6, для случайного леса max_depth составила 9, n_estimators - 90. Было установлено, что только модель случайного леса будет допущена до тестирования, так как только у неё метрика f1 (0.6386) превышает заданный порог. (У дерева решений f1 = 0.5831). 

**Метод увеличения выборки**

In [26]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

In [27]:
#for depth in range(2, 18):
balanced_model = DecisionTreeClassifier(
random_state = 12345, max_depth = 7)
balanced_model.fit(features_upsampled, target_upsampled)
predicted_valid = balanced_model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print("f1", ":", round(f1, 4))
#    print("Глубина дерева", depth, ":", f1)

f1 : 0.5297


In [28]:
probabilities_valid = balanced_model.predict_proba(features_valid) 
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(round(auc_roc,4))

0.8254


In [29]:
balanced_model = RandomForestClassifier(n_estimators = 50, random_state = 12345, max_depth = 20)
balanced_model.fit(features_upsampled, target_upsampled)
predicted_valid = balanced_model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print("f1", ":", round(f1, 4))

f1 : 0.6109


In [30]:
probabilities_valid = balanced_model.predict_proba(features_valid) 
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(round(auc_roc,4))

0.8618


Таким образом, метод увеличения выборки также демонстрирует, что f1 критерий у модели случайного леса выше чем у модели дерево решений, поэтому в дальнейшим данный метод будет использован на тестовой выборке. Что касается показателя roc_auc, то у модели дерево решений он составил 0.8254, а у модели случайного леса - 0.8618.

## Тестирование модели

In [31]:
model = RandomForestClassifier(n_estimators = 90, random_state = 12345, max_depth = 9,  class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
f1 = f1_score(target_test, predicted_test)
print('f1:', round(f1,4))

f1: 0.5907


In [32]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]


auc_roc = roc_auc_score(target_test, probabilities_one_test)
print(round(auc_roc,4))

0.8475


На этапе тестирования f1 - критерий составил 0,5907, что полностью удовлетворяет условиям поставленной задачи. Что касается параметра auc_roc = 0,8475, его можно считать удовлетворительным. 

**Вывод**: Основная цель исследования состояла в создании системы, которая сможет спрогнозировать уйдёт ли клиент в близжайшее время из банка или нет. В качестве исходных данных анализа используется статистика, собранная «Бета-банк». Набор данных содержит информацию о персональнных данных клиента: фамилия, возраст, пол, страна проживания, кредитный рейтинг, сколько лет человек является клиентом банка, наличие кредитной карты, оценка активности потребителя, а также информация об его финансовом состоянии.

В начале проекта была проведена подготовка данных. В столбце Tenure были удалины пропущенные значения, они составляли 10% от всей выборки, поэтому их удаление не привело бы к ухудшению качества модели. Также в этом столбеце была замена типа данных на int. В ходе работы были удалены столбцы, которые не нужны для создания модели такие как: RowNumber, CustomerId, Surname. Все названия столбцов были приведены к змеиному регистру. В конце этапа подготовки данные были разбиты на обучающую, валидационную и тестовую выборку в соотношении 3:1:1.

На этапе исследование задачи было решено использовать две модели: случайный лес и дерево решений. Было установлено, что 
для несбалансированной выборки при гиперпараметрах n_estimators = 50 и max_depth = 20 модель случайного леса продемонстрировала лучшее показатели метрики f1 и auc_roc - 0.6234 и 0.8654 соотвественно. Другая модель показала значение чуть ниже: f1 = 0.5815, auc_roc = 0.8245 при гиперпараметре max_depth = 7. Только модель случайного леса превысила минимальный митрики f1, который по условию составлял 0.59.

На этапе борьбы с дисбалансом было выявлено, что соотношение классов несбалансированно и составляет примерно 80 / 20.
Поэтому в дальнейшем было проведено тестирование моделей с выбранными гиперпараметрами с учётом баланса классов. Использовался метод взвешивания классов и увелечения выборки. В ходе ислодования было установлено, что в обоих случаях модельслучайного леса демонстрировала лучшие показатели и привышала заданный минимальный порог f1.

Именно поэтому, модель случайного леса была выбрана в качестве самой эффективной системы, способной проанализировать поведение клиентов и спрогнозировать уйдёт ли клиент из банка или нет. Данная модель была успешно проверена на тестовой выборке с учетом дисбаланса данных. f1 - критерий составил 0,5907, что полностью удовлетворяет условиям поставленной задачи. Что касается параметра auc_roc = 0,8475, его можно считать удовлетворительным. 